In [38]:
import pandas as pd
import numpy as np
import warnings

#------Читаем файл продаж за последний месяц ---------------------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df = pd.read_excel('MouthSales.XLSX', engine="openpyxl")
    
#------удаляем их header, сдвигаем начало df на один вниз----------

df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True)

#-----переименовываем колонки так, чтобы там не было 2 раза'шт '--
print('----Before----')
print(df.columns)

df.columns = ['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'Поступило', 'Заказано штук', 'Сумма заказов минус комиссия WB, руб.',
       'Выкупили, шт.', 'К перечислению за товар, руб.',
       'Текущий остаток, шт.']
print('----After----')
print(df.columns)


----Before----
Index(['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'шт.', 'шт.', 'Сумма заказов минус комиссия WB, руб.',
       'Выкупили, шт.', 'К перечислению за товар, руб.',
       'Текущий остаток, шт.'],
      dtype='object', name=0)
----After----
Index(['Бренд', 'Предмет', 'Сезон', 'Коллекция', 'Наименование',
       'Артикул продавца', 'Артикул WB', 'Баркод', 'Размер', 'Контракт',
       'Склад', 'Поступило', 'Заказано штук',
       'Сумма заказов минус комиссия WB, руб.', 'Выкупили, шт.',
       'К перечислению за товар, руб.', 'Текущий остаток, шт.'],
      dtype='object')


In [34]:
#-------Grouping by Артикул продавца -----
df_pivot = df.pivot_table(values=['Заказано штук'], index='Артикул продавца', aggfunc = 'sum').sort_values(
                                      by=['Заказано штук'], ascending=False)
#-------let's find speed of buying -------
days = 30
df_pivot['Speed'] = round((df_pivot['Заказано штук'] / 30), 2)


In [35]:
#-----let's find left goods minus goods that are on the way ------

with warnings.catch_warnings(record=True):
    warnings.simplefilter("always")
    df_left = pd.read_excel('left.xlsx', engine="openpyxl")
    
df_left_pivot = df_left.pivot_table(
    values=['Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], 
    index='Артикул продавца', 
    aggfunc = 'sum')

df_left_pivot['Left'] = df_left_pivot['Итого по складам'] - df_left_pivot['Товары в пути до клиента'] - df_left_pivot['Товары в пути от клиента']


In [36]:
#----------------Combining two tables -----------------
result = pd.merge(df_pivot, df_left_pivot, on="Артикул продавца")
result.drop(columns = ['Заказано штук', 'Итого по складам', 'Товары в пути до клиента', 'Товары в пути от клиента'], axis = 1, 
            inplace = True) 

In [37]:
#------------- How much to deliver so it is okey for next 60 days  --------
result['To deliver'] = result['Speed'] * 60 + 10 - result['Left']

#------------- Find a week product ----------------------------------------
result['Weak product'] = np.where(result['Speed'] > 0.5, False, True)


In [33]:
result.to_excel("To deliver.xlsx") 